In [134]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Import Libraries**

In [135]:
import pandas as pd
import heapq
from datetime import datetime

example_clients_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/example/input_clients.csv")
example_instruments_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/example/input_instruments.csv")
example_orders_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/example/input_orders.csv")

test_clients_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/test/input_clients.csv")
test_instruments_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/test/input_instruments.csv")
test_orders_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/test/input_orders.csv")

## **Output Dataframes**

In [136]:
exchange_report_data = {'Order Id': [], 'Rejection Reason': []}
exchange_report_df = pd.DataFrame(exchange_report_data)
exchange_report_df

,Order Id,Rejection Reason


## **Preprocessing Dataset**

### Open Auction

In [137]:
test_orders_converted = test_orders_df.copy()
test_orders_converted['Time'] = pd.to_datetime(test_orders_converted['Time'], format='%H:%M:%S', errors='coerce')
test_orders_converted = test_orders_converted[test_orders_converted['Time'].dt.time < pd.to_datetime('09:30:00').time()]
test_orders_converted

,Time,OrderID,Instrument,Quantity,Client,Price,Side
0,1900-01-01 09:00:00,ORDER_10,INST_000,17500.0,CLIENT_107,Market,Sell
1,1900-01-01 09:00:00,ORDER_1430,INST_000,3000.0,CLIENT_111,Market,Sell
2,1900-01-01 09:00:00,ORDER_1860,INST_000,46000.0,CLIENT_113,Market,Sell
3,1900-01-01 09:00:00,ORDER_20,INST_001,4350.0,CLIENT_106,Market,Sell
4,1900-01-01 09:00:00,ORDER_2430,INST_001,8700.0,CLIENT_103,Market,Buy
...,...,...,...,...,...,...,...
895,1900-01-01 09:29:00,ORDER_9459,INST_008,52000.0,CLIENT_108,100.1,Buy
896,1900-01-01 09:29:00,ORDER_9889,INST_008,84000.0,CLIENT_111,100.7,Buy
897,1900-01-01 09:29:00,ORDER_1029,INST_009,15000.0,CLIENT_112,101.0,Sell
898,1900-01-01 09:29:00,ORDER_10459,INST_009,79000.0,CLIENT_108,99.7,Sell


### Check Instrument

In [138]:
instrument_condition = test_orders_df['Instrument'].isin(test_instruments_df['InstrumentID'])
frames = [test_orders_df, instrument_condition]
result = pd.concat(frames,axis=1, join='inner')
cols=pd.Series(result.columns)

for dup in cols[cols.duplicated()].unique():
    cols[cols[cols == dup].index.values.tolist()] = [dup + '.' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]

# rename the columns with the cols list.
result.columns=cols
filtered_result = result[result['Instrument.1']==False]
filtered_result

,Time,OrderID,Instrument,Quantity,Client,Price,Side,Instrument.1


## **Classes**

In [139]:
class Instrument:
    def __init__(self, instrumentID, currency, lotsize):
      self.instrumentID = instrumentID
      self.currency = currency
      self.lotsize = lotsize
      self.openprice = 0
      self.closeprice = 0
      self.totaltradedvolume = 0
      self.dayhigh = 0
      self.daylow = 0
      self.volumeweightedaverageprice = 0
      self.timeobject = None

In [140]:
class Order:
  def __init__(self, order_id, timestamp, instrument, price, quantity, client,side):
      self.order_id = order_id
      self.timestamp = timestamp
      self.instrument = instrument
      self.price = price
      self.quantity = quantity
      self.client = client
      self.side = side  # 'buy' or 'sell'

  def __lt__(self, other):
      # Define comparison for order objects (to be used in priority queue)
      # Market orders are considered to have infinite priority
      if self.price == 'Market':
          return True
      elif other.price == 'Market':
          return False
      else:
          return self.price < other.price

In [141]:
class Client:
    def __init__(self, clientID, currencies, position_check, rating):
      self.clientID = clientID
      self.currencies = currencies
      self.position_check = position_check
      self.rating = rating


In [142]:
class OpenAuction:
    def __init__(self, positions):
        self.orders_heap = []
        self.positions = positions
        self.match_price = 0


    def add_order(self, orders):
        for order in orders:
            heapq.heappush(self.orders_heap, order)

    def match_orders(self, positions):
        market_orders = []
        limit_orders = []
        match_prices = []
        match_orders = []

        # Separate market and limit orders
        while self.orders_heap:
            order = heapq.heappop(self.orders_heap)
            if order.price == 'Market':
                market_orders.append(order)
            else:
                limit_orders.append(order)

        # Match market orders with limit orders
        for market_order in market_orders:
            while limit_orders:
                limit_order = heapq.heappop(limit_orders)
                if market_order.side != limit_order.side:
                    matched_quantity = min(market_order.quantity, limit_order.quantity)

                    if market_order.side == 'Buy':
                      positions[market_order.client][market_order.instrument] +=  matched_quantity
                      positions[limit_order.client][limit_order.instrument] -=  matched_quantity

                    if market_order.side == 'Sell':
                      positions[limit_order.client][limit_order.instrument] +=  matched_quantity
                      positions[limit_order.client][limit_order.instrument] -=  matched_quantity

                    print(f"Matched order: {market_order.order_id} ({market_order.instrument}) and {limit_order.order_id} ({limit_order.instrument}) for {matched_quantity} at price {limit_order.price}")
                    match_prices.append(limit_order.price)
                    match_orders.append(market_order.order_id)
                    match_orders.append(limit_order.order_id)

                    market_order.quantity -= matched_quantity
                    limit_order.quantity -= matched_quantity
                    if limit_order.quantity > 0:
                        heapq.heappush(limit_orders, limit_order)
                    if market_order.quantity == 0:
                        break

            if market_order.quantity > 0:
                print(f"Unmatched market order: {market_order.order_id} ({market_order.instrument}) for {market_order.quantity} at market price")

        self.match_price = sorted(match_prices,reverse=True)[0]
        return (self.match_price,positions)


## **Open Auction Functions**

In [143]:
def create_order_objects(data):

    orders_list = []

    for index, row in data.iterrows():
        timestamp = row['Time']
        order_id = row['OrderID']
        instrument = row['Instrument']
        quantity = row['Quantity']
        client = row['Client']
        price = row['Price']
        side = row['Side']

        order = Order(order_id,timestamp, instrument, price,quantity, client,side)

        orders_list.append(order)


    return orders_list

In [144]:
orders = create_order_objects(test_orders_converted)

In [145]:
def create_instrument_objects(data):

    instruments_list = []

    for index, row in data.iterrows():

        instrumentID = row['InstrumentID']
        currency = row['Currency']
        lotsize = row['LotSize']

        instrument = Instrument(instrumentID, currency, lotsize)

        instruments_list.append(instrument)


    return set(instruments_list)

In [146]:
instruments_list = create_instrument_objects(test_instruments_df)

In [147]:
def create_positions(orders, test_instruments_list):
  client_position = {}
  for client in orders:
    client_position[client.client] = {}
    for instrument in test_instruments_list:
      client_position[client.client][instrument.instrumentID] = 0
  return client_position

In [148]:
positions = create_positions(orders, instruments_list)
positions

In [150]:
open_auction = OpenAuction(positions)
orders_filtered = create_order_objects(test_orders_converted)
open_auction.add_order(orders_filtered)
open_auction.match_orders(positions)

Matched order: ORDER_10880 (INST_009) and ORDER_1868 (INST_000) for 7000.0 at price 100.0
Matched order: ORDER_10450 (INST_009) and ORDER_87 (INST_007) for 68.0 at price 100.0
Matched order: ORDER_10450 (INST_009) and ORDER_8431 (INST_007) for 72.0 at price 100.0
Matched order: ORDER_10450 (INST_009) and ORDER_426 (INST_003) for 67.0 at price 100.0
Matched order: ORDER_10450 (INST_009) and ORDER_2439 (INST_001) for 8400.0 at price 100.0
Matched order: ORDER_10450 (INST_009) and ORDER_2431 (INST_001) for 3900.0 at price 100.0
Matched order: ORDER_10450 (INST_009) and ORDER_7869 (INST_006) for 61.0 at price 100.0
Matched order: ORDER_10450 (INST_009) and ORDER_8888 (INST_007) for 86.0 at price 100.0
Matched order: ORDER_10450 (INST_009) and ORDER_94 (INST_008) for 41000.0 at price 100.0
Matched order: ORDER_10450 (INST_009) and ORDER_9879 (INST_008) for 56000.0 at price 100.0
Matched order: ORDER_10450 (INST_009) and ORDER_3443 (INST_002) for 85.0 at price 100.0
Matched order: ORDER_1045

('100.6',
 {'CLIENT_107': {'INST_001': 0,
   'INST_005': -40000.0,
   'INST_009': 0,
   'INST_002': 152.0,
   'INST_004': 0,
   'INST_003': 0,
   'INST_000': 0,
   'INST_007': 0,
   'INST_008': -112213.0,
   'INST_006': 0},
  'CLIENT_111': {'INST_001': 9400.0,
   'INST_005': -9172.0,
   'INST_009': 0.0,
   'INST_002': -507.0,
   'INST_004': 0.0,
   'INST_003': -152.0,
   'INST_000': 0,
   'INST_007': -54.0,
   'INST_008': 0.0,
   'INST_006': 0},
  'CLIENT_113': {'INST_001': 0,
   'INST_005': 0.0,
   'INST_009': 168000.0,
   'INST_002': 0.0,
   'INST_004': 0,
   'INST_003': 156.0,
   'INST_000': 0.0,
   'INST_007': 0,
   'INST_008': 0,
   'INST_006': 74.0},
  'CLIENT_106': {'INST_001': 0,
   'INST_005': 0.0,
   'INST_009': 0,
   'INST_002': 104.0,
   'INST_004': 0,
   'INST_003': 0,
   'INST_000': 0.0,
   'INST_007': 0.0,
   'INST_008': 0.0,
   'INST_006': 0},
  'CLIENT_103': {'INST_001': 17400.0,
   'INST_005': 28000.0,
   'INST_009': 0.0,
   'INST_002': -232.0,
   'INST_004': 0.0,
   

In [151]:
open_auction.match_price

'100.6'

In [152]:
order_instrument_dict = {}
for new_order in orders_filtered:
  if new_order.instrument not in order_instrument_dict:
    order_instrument_dict[new_order.instrument] = {"current_price":0,"bid":[],"ask":[]}
    order_instrument_dict[new_order.instrument]['current_price'] = open_auction.match_price

  if new_order.quantity == 0:
    continue

  if new_order.side == 'Sell':
    order_instrument_dict[new_order.instrument]['ask'].append(new_order)

  else:
    order_instrument_dict[new_order.instrument]['bid'].append(new_order)

In [153]:
# order_instrument_dict

In [154]:
# positions

## **Continuous Trading**

In [155]:
import pandas as pd
from datetime import datetime

example_clients_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/example/input_clients.csv")
example_instruments_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/example/input_instruments.csv")
example_orders_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/example/input_orders.csv")

test_clients_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/test/input_clients.csv")
test_instruments_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/test/input_instruments.csv")
test_orders_df = pd.read_csv("/content/drive/MyDrive/BoA 2024/Data/test/input_orders.csv")

test_orders_list = list(map(lambda x:Order(order_id=x[1], timestamp=x[0], instrument=x[2], price=x[5], quantity=x[3], side=x[6], client=x[4]), test_orders_df.values.tolist()))
# test_orders_list = list(map(lambda x:Order(order_id=x[1], timestamp=x[0], instrument=x[2], price=x[5], quantity=x[3], side=x[6], client=x[4]), example_orders_df.values.tolist()))
for index, ele in enumerate(test_orders_list):
  test_orders_list[index].timeobject = datetime.strptime(test_orders_list[index].timestamp, '%H:%M:%S')
test_orders_list.sort(key=lambda order: order.timeobject)

test_instruments_list = list(map(lambda x:Instrument(instrumentID=x[0], currency=x[1], lotsize=x[2]), test_instruments_df.values.tolist()))
# test_instruments_list = list(map(lambda x:Instrument(instrumentID=x[0], currency=x[1], lotsize=x[2]), example_instruments_df.values.tolist()))
test_instruments_dict = {}
for instru in test_instruments_list:
  test_instruments_dict[instru.instrumentID] = instru

test_clients_list = list(map(lambda x:Client(clientID=x[0], currencies=x[1], position_check=x[2], rating=x[3]), test_clients_df.values.tolist()))
# test_clients_list = list(map(lambda x:Client(clientID=x[0], currencies=x[1], position_check=x[2], rating=x[3]), example_clients_df.values.tolist()))
test_client_dict = {}
for client in test_clients_list:
  test_client_dict[client.clientID] = client

invalid_orders = {"Order Id": [], "Rejection Reason": []}

# Filter out orders before 930am..


# ask list sorted by price, lowest to highest
# bid list sorted by price, highest to lowest
def sort_price_list(curr_list, reverse=False):
  return sorted(curr_list, key=lambda ele: ele.price, reverse=reverse)

client_position = {}
for client in test_clients_list:
  client_position[client.clientID] = {}
  for instrument in test_instruments_list:
    client_position[client.clientID][instrument.instrumentID] = 0

order_instrument_dict = {}
for new_order in test_orders_list:
  # Policy check
  if new_order.instrument not in test_instruments_dict:
    invalid_orders["Order Id"].append(new_order.order_id)
    invalid_orders["Rejection Reason"].append("REJECTED – INSTRUMENT NOT FOUND")
    continue
  # Currency check
  current_client = test_client_dict[new_order.client]
  client_currencies = current_client.currencies.split(",")
  if test_instruments_dict[new_order.instrument].currency not in client_currencies:
    invalid_orders["Order Id"].append(new_order.order_id)
    invalid_orders["Rejection Reason"].append("REJECTED – MISMATCH CURRENCY")
    continue

  if new_order.instrument not in order_instrument_dict:
    order_instrument_dict[new_order.instrument] = {"current_price":0,"bid":[],"ask":[]}
    # Skip market order since no previous prices
    if new_order.price == "Market":
      order_instrument_dict[new_order.instrument]["bid" if new_order.side == "Buy" else "ask"].append(new_order)
      continue

  # Check lot size
  if new_order.quantity % test_instruments_dict[new_order.instrument].lotsize != 0:
    invalid_orders["Order Id"].append(new_order.order_id)
    invalid_orders["Rejection Reason"].append("REJECTED – INVALID LOT SIZE")
    continue

  bought = False
  if new_order.side == "Buy":
    # Check existing ask orders
    while not bought and len(order_instrument_dict[new_order.instrument]["ask"]) > 0:
      ask_order = order_instrument_dict[new_order.instrument]["ask"].pop(0)
      # Can buy?
      # print(ask_order.quantity, new_order.price)
      if new_order.price != "Market":
        if ask_order.price != "Market" and ask_order.price > new_order.price:
          order_instrument_dict[new_order.instrument]["ask"].append(ask_order)
          break

      # Check the quantity
      # print("BUYING")
      if new_order.quantity <= ask_order.quantity:
        bought = True
        client_position[new_order.client][ask_order.instrument] += new_order.quantity # Add into position
        client_position[ask_order.client][ask_order.instrument] -= new_order.quantity # Add into position
        ask_order.quantity -= new_order.quantity
        # Add back if still have
        if ask_order.quantity > 0:
          order_instrument_dict[new_order.instrument]["ask"].append(ask_order)
      else:
        new_order.quantity -= ask_order.quantity
        client_position[new_order.client][ask_order.instrument] += ask_order.quantity # Add into position
        client_position[ask_order.client][ask_order.instrument] -= ask_order.quantity # Add into position

    # Cannot buy, add into bids
    if not bought:
      order_instrument_dict[new_order.instrument]["bid"].append(new_order)
  else:
    # Check existing bid orders
    for bid_index in range(len(order_instrument_dict[new_order.instrument]["bid"])):
      bid_order = order_instrument_dict[new_order.instrument]["bid"][bid_index]

      if new_order.price != "Market":
        # Can sell?
        if bid_order.price != "Market" and bid_order.price < new_order.price:
          continue
        # Cannot short sell
        if new_order.instrument not in client_position[new_order.client] and test_client_dict[new_order.client].position_check == "Y":
          invalid_orders["Order Id"].append(new_order.order_id)
          invalid_orders["Rejection Reason"].append("REJECTED – POSITION CHECK FAILED")
          continue

      # print("SELLING")
      # Check the quantity
      if new_order.quantity <= bid_order.quantity:
        bought = True
        client_position[new_order.client][bid_order.instrument] -= new_order.quantity # Add into position
        client_position[bid_order.client][bid_order.instrument] += new_order.quantity # Add into position
        bid_order.quantity -= new_order.quantity
        # Remove bid order
        if bid_order.quantity == 0:
          order_instrument_dict[new_order.instrument]["bid"].pop(bid_index)
        else:
          # Update element
          order_instrument_dict[new_order.instrument]["bid"][bid_index] = bid_order
      else:
        new_order.quantity -= bid_order.quantity
        # remove bid order
        order_instrument_dict[new_order.instrument]["bid"].pop(bid_index)
        client_position[new_order.client][bid_order.instrument] -= bid_order.quantity # Add into position
        client_position[bid_order.client][bid_order.instrument] += bid_order.quantity # Add into position
      break

    # Cannot buy, add into ask
    if not bought:
      order_instrument_dict[new_order.instrument]["ask"].append(new_order)

  # Sort bid and ask lists
  order_instrument_dict[new_order.instrument]["bid"] = sort_price_list(order_instrument_dict[new_order.instrument]["bid"])
  order_instrument_dict[new_order.instrument]["ask"] = sort_price_list(order_instrument_dict[new_order.instrument]["ask"], reverse=True)

# print(order_instrument_dict)
# print(client_position)
print(invalid_orders)

# A 2300
# B -5800
# C 4300
# E -800

{'Order Id': ['ORDER_10', 'ORDER_20', 'ORDER_30', 'ORDER_40', 'ORDER_50', 'ORDER_60', 'ORDER_70', 'ORDER_80', 'ORDER_90', 'ORDER_100', 'ORDER_1875', 'ORDER_2875', 'ORDER_3875', 'ORDER_4875', 'ORDER_5875', 'ORDER_6875', 'ORDER_7875', 'ORDER_8875', 'ORDER_9875', 'ORDER_10875', 'ORDER_1450', 'ORDER_2450', 'ORDER_3450', 'ORDER_4450', 'ORDER_5450', 'ORDER_6450', 'ORDER_7450', 'ORDER_8450', 'ORDER_9450', 'ORDER_10450', 'ORDER_125', 'ORDER_1455', 'ORDER_225', 'ORDER_2455', 'ORDER_325', 'ORDER_3455', 'ORDER_425', 'ORDER_4455', 'ORDER_525', 'ORDER_5455', 'ORDER_625', 'ORDER_6455', 'ORDER_725', 'ORDER_7455', 'ORDER_825', 'ORDER_8455', 'ORDER_925', 'ORDER_9455', 'ORDER_1025', 'ORDER_10455', 'ORDER_135', 'ORDER_235', 'ORDER_335', 'ORDER_435', 'ORDER_535', 'ORDER_635', 'ORDER_735', 'ORDER_835', 'ORDER_935', 'ORDER_1035', 'ORDER_1900', 'ORDER_2900', 'ORDER_3900', 'ORDER_4900', 'ORDER_5900', 'ORDER_6900', 'ORDER_7900', 'ORDER_8900', 'ORDER_9900', 'ORDER_10900', 'ORDER_1475', 'ORDER_2475', 'ORDER_3475

## **Closing Auction**

In [156]:
test_orders_converted_close = test_orders_df.copy()
test_orders_converted_close['Time'] = pd.to_datetime(test_orders_converted_close['Time'], format='%H:%M:%S', errors='coerce')
test_orders_converted_close = test_orders_converted_close[test_orders_converted_close['Time'].dt.time > pd.to_datetime('16:00:00').time()]
test_orders_converted_close

,Time,OrderID,Instrument,Quantity,Client,Price,Side
9820,1900-01-01 16:01:00,ORDER_1421,INST_000,99000.0,CLIENT_104,99.7,Sell
9821,1900-01-01 16:01:00,ORDER_1851,INST_000,95000.0,CLIENT_103,100.7,Buy
9822,1900-01-01 16:01:00,ORDER_2421,INST_001,1400.0,CLIENT_104,100.2,Sell
9823,1900-01-01 16:01:00,ORDER_2851,INST_001,8900.0,CLIENT_105,100.1,Buy
9824,1900-01-01 16:01:00,ORDER_3421,INST_002,100.0,CLIENT_108,100.8,Sell
...,...,...,...,...,...,...,...
9995,1900-01-01 16:09:00,ORDER_8859,INST_007,62.0,CLIENT_104,99.6,Sell
9996,1900-01-01 16:09:00,ORDER_9429,INST_008,59000.0,CLIENT_112,100.1,Buy
9997,1900-01-01 16:09:00,ORDER_9859,INST_008,94000.0,CLIENT_109,100.9,Buy
9998,1900-01-01 16:09:00,ORDER_10429,INST_009,31000.0,CLIENT_105,100.1,Sell


In [157]:
test_orders_close = create_order_objects(test_orders_converted_close)
test_orders_close

## **Export to CSV**

In [161]:
exchange_report = pd.DataFrame(invalid_orders)
exchange_report.to_csv('/content/drive/MyDrive/BoA 2024/output_exchange_report.csv', index=False)

In [163]:
client_report = pd.DataFrame(client_position)
client_report.to_csv('/content/drive/MyDrive/BoA 2024/output_client_report.csv', index=False)